# vTTS - GPT-SoVITS v3 음성 클로닝 테스트

**Zero-shot Voice Cloning** - 참조 오디오 하나로 음성 클로닝!

## 🎯 GPT-SoVITS의 핵심

- **Zero-shot**: 5~10초 참조 오디오만으로 음성 클로닝
- **다국어**: 중국어, 영어, 일본어, 한국어, 광동어 지원
- **고품질**: v3/v4 최신 모델로 자연스러운 음성

⚠️ **중요**: GPT-SoVITS는 `reference_audio`와 `reference_text`가 **필수**입니다!

이 노트북에서는:
- ✅ **Part 0**: GPT-SoVITS 설치 및 환경 설정
- ✅ **Part 1**: 참조 오디오 준비
- ✅ **Part 2**: 음성 클로닝 테스트
- ✅ **Part 3**: Python 클라이언트 사용

## 🚀 Part 0: 설치 및 환경 설정

In [ ]:
# GPU 확인
import torch

print(f"CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
    print(f"CUDA Version: {torch.version.cuda}")
    DEVICE = "cuda"
else:
    print("⚠️ GPU 없음 - GPT-SoVITS는 GPU 권장!")
    DEVICE = "cpu"

In [ ]:
# GPT-SoVITS 설치 (시간이 오래 걸립니다 ~10-15분)
import os
import subprocess
import sys

print("🔧 GPT-SoVITS 환경 설정 중...")
print("   ⚠️ 이 과정은 10-15분 정도 소요됩니다.\n")

# 1. numpy 호환성
print("   [1/6] numpy 호환성 처리...")
!pip uninstall numpy -y -q
!pip install -q "numpy>=1.24.0,<2.0.0"

# 2. vTTS + Supertonic 설치 (GPU 자동 지원)
print("   [2/6] vTTS + Supertonic 설치 (GPU 자동 지원)...")
!pip install -q "vtts[supertonic] @ git+https://github.com/bellkjtt/vTTS.git"

# 3. CUDA 지원 확인
print("   [3/6] CUDA 지원 확인...")
import onnxruntime as ort
providers = ort.get_available_providers()
print(f"      ONNX Providers: {providers}")
if "CUDAExecutionProvider" in providers:
    print("      🎉 CUDA 지원 활성화!")

# 4. GPT-SoVITS 저장소 클론
print("   [4/6] GPT-SoVITS 저장소 클론...")
GPT_SOVITS_PATH = os.path.expanduser("~/.vtts/GPT-SoVITS")
os.makedirs(os.path.dirname(GPT_SOVITS_PATH), exist_ok=True)

if not os.path.exists(GPT_SOVITS_PATH):
    !git clone --depth 1 https://github.com/RVC-Boss/GPT-SoVITS.git {GPT_SOVITS_PATH}
else:
    print(f"      GPT-SoVITS already exists: {GPT_SOVITS_PATH}")

# 5. GPT-SoVITS requirements 설치
print("   [5/6] GPT-SoVITS 의존성 설치 (오래 걸림)...")
!pip install -q -r {GPT_SOVITS_PATH}/requirements.txt

# 6. 환경변수 설정
print("   [6/6] 환경변수 설정...")
os.environ["GPT_SOVITS_PATH"] = GPT_SOVITS_PATH

print(f"\n✅ GPT-SoVITS 설치 완료!")
print(f"   경로: {GPT_SOVITS_PATH}")

## 📁 Part 1: 참조 오디오 준비

GPT-SoVITS는 **참조 오디오**가 필수입니다!

- 권장 길이: 3~10초
- 깨끗한 음성 (배경 소음 없음)
- 참조 텍스트: 참조 오디오에서 말하는 내용

In [ ]:
# 참조 오디오 생성 (Supertonic 사용)
import os
import soundfile as sf

REF_DIR = "./reference_audio"
os.makedirs(REF_DIR, exist_ok=True)

print("🎙️ 참조 오디오 생성 중 (Supertonic 사용)...\n")

try:
    from vtts.engines.supertonic import SupertonicEngine
    from vtts.engines.base import TTSRequest
    
    # Supertonic 엔진 로드
    engine = SupertonicEngine(model_id="Supertone/supertonic-2", device=DEVICE)
    engine.load_model()
    
    # 참조 텍스트 (중요: 이 텍스트가 참조 오디오의 내용!)
    REF_TEXT_KO = "안녕하세요, 저는 음성 클로닝을 위한 참조 음성입니다."
    REF_TEXT_EN = "Hello, this is a reference audio for voice cloning."
    
    # 한국어 참조 오디오 생성
    request_ko = TTSRequest(
        text=REF_TEXT_KO,
        language="ko",
        voice="F1",
        speed=1.0,
        extra_params={"total_steps": 10}
    )
    output_ko = engine.synthesize(request_ko)
    
    REF_AUDIO_KO = os.path.join(REF_DIR, "reference_ko.wav")
    sf.write(REF_AUDIO_KO, output_ko.audio, output_ko.sample_rate)
    print(f"✅ 한국어 참조 오디오: {REF_AUDIO_KO}")
    print(f"   내용: {REF_TEXT_KO}")
    
    # 영어 참조 오디오 생성
    request_en = TTSRequest(
        text=REF_TEXT_EN,
        language="en",
        voice="F1",
        speed=1.0,
        extra_params={"total_steps": 10}
    )
    output_en = engine.synthesize(request_en)
    
    REF_AUDIO_EN = os.path.join(REF_DIR, "reference_en.wav")
    sf.write(REF_AUDIO_EN, output_en.audio, output_en.sample_rate)
    print(f"\n✅ 영어 참조 오디오: {REF_AUDIO_EN}")
    print(f"   내용: {REF_TEXT_EN}")
    
    engine.unload_model()
    print("\n✅ 참조 오디오 준비 완료!")
    
except Exception as e:
    print(f"⚠️ 참조 오디오 생성 실패: {e}")
    print("   직접 오디오 파일을 업로드하세요!")
    REF_AUDIO_KO = None
    REF_AUDIO_EN = None